# 工具集

In [2]:
# Tool webloader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import TextLoader


llm = OpenAI(temperature=0)
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

 
embeddings = OpenAIEmbeddings()
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever())



# Tool search
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key="put your key here")
search_tool = Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )

Created a chunk of size 2134, which is longer than the specified 1000
Created a chunk of size 1915, which is longer than the specified 1000
Created a chunk of size 1091, which is longer than the specified 1000
Created a chunk of size 1543, which is longer than the specified 1000
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


KeyboardInterrupt: 

# AgentType
经测试ZERO_SHOT_REACT_DESCRIPTION即可很好满足需要，同时只要将search工具放在最后一个，就能实现先调用已有工具最后再使用search，反之则会无脑先使用Search。

In [4]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI


llm = OpenAI(temperature=0)


tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = "Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question."
    ),
]

# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

#询问指定文件内容
agent.run("介绍一下Ruff?")




> Entering new AgentExecutor chain...
 我需要找到Ruff的相关信息
Action: Search
Action Input: Ruff
Observation: {'type': 'dictionary_results', 'syllables': 'ruff', 'phonetic': '/rəf/', 'word_type': 'noun'}
Thought: 我需要更多的信息
Action: Ruff QA System
Action Input: What is Ruff?
Observation:  Ruff is a linter and formatter for Python code written in Rust. It is compatible with Flake8 plugins, but uses different rule codes and prefixes to minimize conflicts. It does not support custom lint rules, but popular Flake8 plugins are re-implemented in Rust as part of Ruff itself.
Thought: 我现在知道最终答案了
Final Answer: Ruff是一个用Rust编写的Python代码的语法检查器和格式化器，它与Flake8插件兼容，但使用不同的规则代码和前缀以最大限度地减少冲突。它不支持自定义lint规则，但是常用的Flake8插件作为Ruff本身的一部分，已经在Rust中重新实现。

> Finished chain.


'Ruff是一个用Rust编写的Python代码的语法检查器和格式化器，它与Flake8插件兼容，但使用不同的规则代码和前缀以最大限度地减少冲突。它不支持自定义lint规则，但是常用的Flake8插件作为Ruff本身的一部分，已经在Rust中重新实现。'

In [5]:
#搜索
agent.run("介绍一下OpenAI Dev Day?")



> Entering new AgentExecutor chain...
 我需要找到一些有关OpenAI Dev Day的信息
Action: Search
Action Input: OpenAI Dev Day
Observation: [{'title': 'Everything announced at OpenAI’s first developer event', 'link': 'https://techcrunch.com/2023/11/06/everything-announced-at-openais-first-developer-event/', 'source': 'TechCrunch', 'date': '2 hours ago', 'thumbnail': 'https://serpapi.com/searches/6549ff08681ebea18151f459/images/0bffca266b3e3f5b139a4590ef4442ab678bff9fab1528a4.jpeg'}, {'title': 'OpenAI creates market for custom ChatGPTs', 'link': 'https://www.axios.com/2023/11/06/open-ai-dev-day-chatgpt-custom', 'source': 'Axios', 'date': '8 hours ago', 'thumbnail': 'https://serpapi.com/searches/6549ff08681ebea18151f459/images/0bffca266b3e3f5bc6df56ba4f2e87c61be5e958fb0abb66.jpeg'}, {'title': 'OpenAI unveils custom GPTs, Store, and Assistant API, at Dev Day', 'link': 'https://www.zdnet.com/article/openai-unveils-custom-gpts-store-and-assistant-api-at-dev-day/', 'source': 'ZDNet', 'date': '10 hours ago'

'OpenAI在其首次开发者日活动中推出了一个新的助手API，让开发人员构建自己的“代理类体验”。开发人员可以使用OpenAI的GPT-4 Turbo模型，以及新的GPT Store，来构建自定义的聊天机器人。'

In [6]:
agent.run("Ruff取代了哪些工具?")



> Entering new AgentExecutor chain...
 I need to find out what tools Ruff replaces.
Action: Search
Action Input: "What tools does Ruff replace?"
Observation: Ruff can be used to replace Flake8 (plus dozens of plugins), isort, pydocstyle, yesqa, eradicate, pyupgrade, and autoflake, all while executing tens or hundreds of times faster than any individual tool. This has the potential to simplify and speed up our code checks massively.
Thought: I now know the final answer
Final Answer: Ruff can replace Flake8 (plus dozens of plugins), isort, pydocstyle, yesqa, eradicate, pyupgrade, and autoflake.

> Finished chain.


'Ruff can replace Flake8 (plus dozens of plugins), isort, pydocstyle, yesqa, eradicate, pyupgrade, and autoflake.'

In [2]:
import gradio as gr
def answer(ask):
    ans=agent.run(ask)
    return ans


ui = gr.Interface(fn=answer, inputs=gr.Textbox(label="请输入你想查询的信息："), outputs="text",title = "<h1 style='font-size: 40px;'><center>知识图谱问答</center></h1>")
ui.launch()

c:\Users\47131\.conda\envs\swanchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\47131\.conda\envs\swanchain\lib\site-packages\gradio\queueing.py", line 427, in call_prediction
    output = await route_utils.call_process_api(
  File "c:\Users\47131\.conda\envs\swanchain\lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\47131\.conda\envs\swanchain\lib\site-packages\gradio\blocks.py", line 1486, in process_api
    result = await self.call_function(
  File "c:\Users\47131\.conda\envs\swanchain\lib\site-packages\gradio\blocks.py", line 1108, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\47131\.conda\envs\swanchain\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\47131\.conda\envs\swanchain\lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "c:\